# Submittin an FL Hub job

In [1]:
# add path
import sys
print(sys.executable)
print(sys.version)

NVFLARE_HOME=f"/home/hroth/Code2/nvflare/flhub"
HUB_EXAMPLE=f"{NVFLARE_HOME}/examples/flhub"
sys.path.append(f"{NVFLARE_HOME}")
sys.path.append(f"{NVFLARE_HOME}/examples")
sys.path.append(f"{NVFLARE_HOME}/integration/monai")

/home/hroth/Code2/nvflare/flhub/examples/flhub/nvflare_flhub/bin/python3.8
3.8.15 (default, Oct 12 2022, 19:15:16) 
[GCC 11.2.0]


In [2]:
# (optional) check environment
!python --version
!pip freeze | grep torch
!pip freeze | grep monai

Python 3.8.15
pytorch-ignite==0.4.10
torch==1.13.0
monai==1.0.1


### Open admin api runner

In [3]:
import msgpack
from nvflare.fuel.hci.client.fl_admin_api_runner import FLAdminAPIRunner, api_command_wrapper

# Initialize the runner
runner = FLAdminAPIRunner(
    username="admin@nvflare.com",
    admin_dir=f"{HUB_EXAMPLE}/workspaces/t1_workspace/admin@nvflare.com",
    poc=False,
    debug=False,
)

### Submit job

In [4]:
job_folder=f"{HUB_EXAMPLE}/job"
api_command_wrapper(runner.api.submit_job(job_folder))

{'status': <APIStatus.SUCCESS: 'SUCCESS'>, 'details': {'message': 'Submitted job: 5d42179a-f65b-4470-bca6-f3cd967b40a4', 'job_id': '5d42179a-f65b-4470-bca6-f3cd967b40a4'}, 'raw': {'time': '2022-11-08 10:49:08.664204', 'data': [{'type': 'string', 'data': 'Submitted job: 5d42179a-f65b-4470-bca6-f3cd967b40a4'}, {'type': 'success', 'data': ''}], 'status': <APIStatus.SUCCESS: 'SUCCESS'>}}


{'status': <APIStatus.SUCCESS: 'SUCCESS'>,
 'details': {'message': 'Submitted job: 5d42179a-f65b-4470-bca6-f3cd967b40a4',
  'job_id': '5d42179a-f65b-4470-bca6-f3cd967b40a4'},
 'raw': {'time': '2022-11-08 10:49:08.664204',
  'data': [{'type': 'string',
    'data': 'Submitted job: 5d42179a-f65b-4470-bca6-f3cd967b40a4'},
   {'type': 'success', 'data': ''}],
  'status': <APIStatus.SUCCESS: 'SUCCESS'>}}

### List jobs

In [5]:
%%capture
jobs = api_command_wrapper(runner.api.list_jobs())

In [6]:
print(jobs["raw"]["data"][0]["data"])

---------------------------------------------------------------------------------------------------------------------------------
| JOB ID                               | NAME          | STATUS             | SUBMIT TIME                      | RUN DURATION   |
---------------------------------------------------------------------------------------------------------------------------------
| dfb2b200-7430-4dd0-9d28-0a9928a6538e | spleen-bundle | FINISHED:COMPLETED | 2022-11-08T10:08:39.813055-05:00 | 0:18:38.519946 |
| 660cd992-f2a5-4142-bf92-d218a59858df | spleen-bundle | FINISHED:COMPLETED | 2022-11-08T10:42:00.855532-05:00 | 0:00:45.770343 |
| 1c332ac8-1206-480f-a34d-8caa5005a908 | spleen-bundle | FINISHED:COMPLETED | 2022-11-08T10:44:09.934393-05:00 | 0:00:47.107029 |
| 5d42179a-f65b-4470-bca6-f3cd967b40a4 | spleen-bundle | SUBMITTED          | 2022-11-08T10:49:08.676352-05:00 | N/A            |
------------------------------------------------------------------------------------------

### Show server log

In [7]:
%%capture
reply = api_command_wrapper(runner.api.tail_target_log("server"))

In [8]:
print(reply["details"]["message"])

2022-11-08 10:44:10,873 - JobRunner - INFO - [identity=t1_project, run=?]: Got the job:1c332ac8-1206-480f-a34d-8caa5005a908 from the scheduler to run
2022-11-08 10:44:10,890 - JobRunner - INFO - [identity=t1_project, run=?]: Application t1_app deployed to the server for job: 1c332ac8-1206-480f-a34d-8caa5005a908
2022-11-08 10:44:10,899 - JobRunner - INFO - [identity=t1_project, run=?]: App t2a_app to be deployed to the clients: t1_client_a for run: 1c332ac8-1206-480f-a34d-8caa5005a908
2022-11-08 10:44:10,906 - JobRunner - INFO - [identity=t1_project, run=?]: App t2b_app to be deployed to the clients: t1_client_b for run: 1c332ac8-1206-480f-a34d-8caa5005a908
2022-11-08 10:44:11,867 - JobRunner - INFO - [identity=t1_project, run=?]: Started run: 1c332ac8-1206-480f-a34d-8caa5005a908 for clients: t1_client_a,t1_client_b
2022-11-08 10:44:14,624 - FederatedServer - INFO - GetTask: Return task: train to client: t1_client_a (bad79a75-a7e2-45fe-aedc-bded6e28290c) 
2022-11-08 10:44:15,038 - Feder

### Show Connector-A log

In [9]:
%%capture
reply = api_command_wrapper(runner.api.tail_target_log("t1_client_a"))

In [10]:
print(reply["details"]["message"])

2022-11-08 10:08:44,074 - ProcessExecutor - INFO - run (dfb2b200-7430-4dd0-9d28-0a9928a6538e): waiting for child worker process to finish.
2022-11-08 10:27:22,501 - ProcessExecutor - INFO - run (dfb2b200-7430-4dd0-9d28-0a9928a6538e): child worker process finished with execution code: 0
2022-11-08 10:42:02,986 - ClientEngine - INFO - Starting client app. rank: 0
2022-11-08 10:42:03,005 - ProcessExecutor - INFO - Worker child process ID: 2466071
2022-11-08 10:42:05,009 - ProcessExecutor - INFO - run (660cd992-f2a5-4142-bf92-d218a59858df): waiting for child worker process to finish.
2022-11-08 10:42:52,765 - ProcessExecutor - INFO - run (660cd992-f2a5-4142-bf92-d218a59858df): child worker process finished with execution code: 0
2022-11-08 10:44:11,712 - ClientEngine - INFO - Starting client app. rank: 0
2022-11-08 10:44:11,735 - ProcessExecutor - INFO - Worker child process ID: 2470502
2022-11-08 10:44:13,756 - ProcessExecutor - INFO - run (1c332ac8-1206-480f-a34d-8caa5005a908): waiting f

### Show Connector-B log

In [11]:
%%capture
reply = api_command_wrapper(runner.api.tail_target_log("t1_client_b"))

In [12]:
print(reply["details"]["message"])

2022-11-08 10:08:44,179 - ProcessExecutor - INFO - run (dfb2b200-7430-4dd0-9d28-0a9928a6538e): waiting for child worker process to finish.
2022-11-08 10:27:22,623 - ProcessExecutor - INFO - run (dfb2b200-7430-4dd0-9d28-0a9928a6538e): child worker process finished with execution code: 0
2022-11-08 10:42:02,985 - ClientEngine - INFO - Starting client app. rank: 0
2022-11-08 10:42:03,005 - ProcessExecutor - INFO - Worker child process ID: 2466066
2022-11-08 10:42:05,010 - ProcessExecutor - INFO - run (660cd992-f2a5-4142-bf92-d218a59858df): waiting for child worker process to finish.
2022-11-08 10:42:52,820 - ProcessExecutor - INFO - run (660cd992-f2a5-4142-bf92-d218a59858df): child worker process finished with execution code: 0
2022-11-08 10:44:11,720 - ClientEngine - INFO - Starting client app. rank: 0
2022-11-08 10:44:11,753 - ProcessExecutor - INFO - Worker child process ID: 2470509
2022-11-08 10:44:13,763 - ProcessExecutor - INFO - run (1c332ac8-1206-480f-a34d-8caa5005a908): waiting f

### Finish

In [13]:
runner.api.logout()

{'status': <APIStatus.ERROR_INACTIVE_SESSION: 'ERROR_INACTIVE_SESSION'>,
 'details': 'session is logging out'}

# Show TensorBoard

In [14]:
# load only the hub server workspace
%load_ext tensorboard
%tensorboard --logdir "/home/hroth/Code2/nvflare/flhub/examples/flhub/workspaces/t1_workspace/localhost"

Note. The tensorboard should also be available at [http://localhost:6006](http://localhost:6006).